In [ ]:
!pip install altair


In [ ]:
import pandas as pd

artistas = pd.read_csv("/content/Artistas.csv", sep=",")
lineup = pd.read_csv("/content/Lineup.csv", sep=",")

In [ ]:
lineup.columns   = lineup.columns.str.strip()
artistas.columns = artistas.columns.str.strip()


In [ ]:
artistas = artistas.rename(columns={
    'Artistas': 'Artista',
    'Género musical': 'Genero_musical'
})

In [ ]:
df = pd.merge(lineup, artistas, on='Artista', how='left')

In [ ]:
df[['Artista','Año','Genero_musical','Composición_y','Género_y']].head()


,Artista,Año,Genero_musical,Composición_y,Género_y
0,311,2011,Rock,Banda,M
1,30 Seconds to Mars,2011,Rock,Banda,M
2,Achu,2011,NaN,NaN,NaN
3,Achu & Los Plumabits,2011,NaN,NaN,NaN
4,Ana Tijoux,2011,NaN,NaN,NaN


In [ ]:
def clasificar_tipo_artista(row):
    comp = row['Composición_x'].strip().lower()
    gen = row['Género_x'].strip().lower()

    if comp == 'solista' and gen == 'masculino':
        return 'Solista Hombre'
    elif comp == 'solista' and gen == 'femenino':
        return 'Solista Mujer'
    elif comp == 'banda' and gen == 'masculino':
        return 'Banda Hombres'
    elif comp == 'banda' and gen == 'femenino':
        return 'Banda Mujeres'
    elif comp == 'banda' and gen == 'mixto':
        return 'Banda Mixta'
    else:
        return 'Desconocido'

df['tipo_artista'] = df.apply(clasificar_tipo_artista, axis=1)

df[['Artista', 'Composición_x', 'Género_x', 'tipo_artista']].head()


,Artista,Composición_x,Género_x,tipo_artista
0,311,Banda,Masculino,Banda Hombres
1,30 Seconds to Mars,Banda,Masculino,Banda Hombres
2,Achu,Banda,Masculino,Banda Hombres
3,Achu & Los Plumabits,Banda,Mixto,Banda Mixta
4,Ana Tijoux,Solista,Femenino,Solista Mujer


In [ ]:
print(df.columns.tolist())


['Año', 'Artista', 'Composición_x', 'Género_x', 'nacionalidad', 'escenario', 'tiempo / escenario', 'headliner', 'Dia', 'Composición_y', 'Género_y', 'Nacionalidad', 'Genero_musical', 'Debut', 'tipo_artista']


In [ ]:
df_agrupado = (
    df.groupby(['Año', 'Genero_musical', 'tipo_artista'])
      .size()
      .reset_index(name='Cantidad')
)

df_agrupado.head()

,Año,Genero_musical,tipo_artista,Cantidad
0,2011,Blues,Solista Hombre,1
1,2011,Cumbia,Banda Hombres,1
2,2011,Electrónica,Banda Hombres,3
3,2011,Electrónica,Solista Hombre,8
4,2011,Folk,Solista Hombre,1


In [ ]:
print(df_agrupado['tipo_artista'].unique())

['Solista Hombre' 'Banda Hombres' 'Solista Mujer' 'Banda Mixta'
 'Banda Mujeres']


In [ ]:
df_agrupado[df_agrupado['tipo_artista'] == 'Banda Mujeres'].head(10)

,Año,Genero_musical,tipo_artista,Cantidad
36,2012,Rock,Banda Mujeres,1
53,2014,Electrónica,Banda Mujeres,1
69,2014,Rock,Banda Mujeres,1
80,2015,Pop,Banda Mujeres,1
88,2015,Rock,Banda Mujeres,1
113,2017,Electrónica,Banda Mujeres,1
170,2019,Rock,Banda Mujeres,1
222,2024,Folk,Banda Mujeres,1


In [ ]:
order = alt.Order('Cantidad:Q', sort='descending')

In [ ]:
import altair as alt

size_scale = alt.Scale(range=[200, 3000])

chart = (
    alt.Chart(df_agrupado)
      .mark_circle(opacity=0.85, stroke='black', strokeWidth=0.4)
      .encode(
          x=alt.X('Año:O',
                  title='Año',
                  axis=alt.Axis(labelAngle=0)),


          y=alt.Y('Genero_musical:N',
                  title='Género musical',
                  scale=alt.Scale(paddingInner=0.45,
                                  paddingOuter=0.15)),


          order=alt.Order('Cantidad:Q', sort='descending'),


          size=alt.Size('Cantidad:Q',
                        scale=size_scale,
                        legend=alt.Legend(title='Cantidad de artistas')),


          color=alt.Color(
              'tipo_artista:N',
              title='Tipo de artista',
              legend=alt.Legend(orient='right'),
              scale=alt.Scale(
                  domain=[
                      "Solista Hombre",
                      "Solista Mujer",
                      "Banda Hombres",
                      "Banda Mujeres",
                      "Banda Mixta"
                  ],
                  range=[
                      "#58b4cb",
                      "#e85343",
                      "#0299f0",
                      "#ed7409",
                      "#c9d849"
                  ]
              )
          ),


          tooltip=[
              alt.Tooltip('Año:O',             title='Año'),
              alt.Tooltip('Genero_musical:N',  title='Género musical'),
              alt.Tooltip('tipo_artista:N',    title='Tipo de artista'),
              alt.Tooltip('Cantidad:Q',        title='Cantidad', format=',d')
          ]
      )
      .properties(
          width=950,
          height=800,
          title='Proporción de artistas y bandas por género musical y tipo (Lollapalooza Chile)'
      )
      .interactive()
)

chart

alt.Chart(...)

In [ ]:
chart.save('bubble_lollapalooza.html')